In [2]:
import pandas as pd
import boto3
from io import StringIO
from botocore.exceptions import ClientError

In [3]:
s3 = boto3.client('s3')

def check_file_exists(bucket_name, file_path):
    try:
        s3.head_object(Bucket=bucket_name, Key=file_path)
        return True
    except ClientError as e:
        # If a 404 error is thrown, the object does not exist
        if e.response['Error']['Code'] == '404':
            return False
        else:
            # Handle other possible exceptions
            raise e

In [9]:
bucket_name = 'otg-fraud-dataml-prod'
file_path = "raw_views/"

In [29]:
response = s3.list_objects_v2(Bucket=bucket_name, 
                              Prefix=file_path)

if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
        # s3.delete_object(Bucket=bucket_name, Key=obj['Key'])
else:
    print("No files found in the path.")


raw_views/v_order_details/year=2023/month=05/v_order_details_2023_05_000.csv
raw_views/v_order_details/year=2023/month=06/v_order_details_2023_06_000.csv
raw_views/v_order_details/year=2023/month=07/v_order_details_2023_07_000.csv
raw_views/v_order_details/year=2023/month=08/v_order_details_2023_08_000.csv
raw_views/v_order_details/year=2023/month=09/v_order_details_2023_09_000.csv
raw_views/v_order_details/year=2023/month=10/v_order_details_2023_10_000.csv
raw_views/v_order_details/year=2023/month=11/v_order_details_2023_11_000.csv
raw_views/v_order_details/year=2023/month=12/v_order_details_2023_12_000.csv
raw_views/v_order_details/year=2024/month=01/v_order_details_2024_01_000.csv
raw_views/v_order_details/year=2024/month=02/v_order_details_2024_02_000.csv
raw_views/v_order_details/year=2024/month=03/v_order_details_2024_03_000
raw_views/v_order_details/year=2024/month=04/v_order_details_2024_04_000.csv
raw_views/v_order_details/year=2024/month=05/v_order_details_2024_05_000.csv
raw

In [ ]:
# s3://otg-fraud-dataml-prod/raw_views/v_order_details/year=2023/month=05/v_order_details_2023_05_000.csv

In [110]:
# file_path = "lacm/inference/15_minutes/year=2024/month=8/day=6/hour=14/minute=6/lacm_000.csv"
file_path = "source_views/v_order_details/year=2023/month=12/v_order_details_2023_12_000.csv"

body = s3.get_object(Bucket=bucket_name, Key=file_path)["Body"].read()
string_data = body.decode("utf-8")
de_data = pd.read_csv(
    StringIO(string_data)
)
print(de_data.shape)
# print(de_data.bus_date.min())
# print(de_data.bus_date.max())

ParserError: Error tokenizing data. C error: Expected 90 fields in line 592333, saw 92


In [100]:
de_data.columns

Index(['country_id', 'bus_date', 'pos_venue_id', 'order_id', 'transaction_id',
       'device_order_id', 'order_event_type_id', 'pos_order_type_id',
       'day_part_id', 'time_zone', 'order_local_time', 'sales_hr',
       'ord_beg_time', 'ord_close_time', 'order_close_server_time',
       'total_time', 'item_count', 'guest_count', 'gross_total', 'net_total',
       'taxes', 'exclusive_tax', 'inclusive_tax', 'food_cost_amt',
       'paper_cost_am', 'total_cost_am', 'reduction_amt', 'reduction_id',
       'promo_id', 'cust_prof_id', 'mobile_ord_fl', 'original_order_id',
       'order_status_id', 'device_id', 'vendor_loc_id', 'custom_desc',
       'fee_amt', 'custom_fee_value', 'mode_id', 'tip_amount', 'tip_status',
       'order_source_id', 'course_code', 'sync_status', 'srce_sys_id',
       'load_ts', 'updt_ts', 'order_modified_time', 'src_sys_id', 'vendor_id',
       'menu_vendor_id', 'pos_order_uuid', 'refund_vendor_id',
       'refund_venue_id', 'pos_terminal_id', 'reduction_cd', 'a

In [ ]:
de_data.shape

In [91]:
de_data.groupby('bus_date')['order_id'].count().values.sum()

1675975

In [83]:
# one_day = de_data[de_data.bus_date.qu('2024-03-01') & de_data.bus_date.lt('2024-03-03')]
one_day = de_data[de_data.bus_date.eq("2024-03-01")]

In [84]:
one_day.shape

(128357, 140)

In [28]:
de_data.bus_date.max()

'2024-01-31'

In [80]:
4681+3859

8540

In [24]:
%%time

body = s3.get_object(Bucket=bucket_name, Key=file_path)["Body"].read()
string_data = body.decode("utf-8")
de_data = pd.read_csv(
    StringIO(string_data)
)
print(de_data.shape)

(38, 140)
CPU times: user 10.8 ms, sys: 10.5 ms, total: 21.3 ms
Wall time: 84.4 ms


In [9]:
de_data.fraud_category.value_counts()

fraud_category
Reduction Fraud                     3889
Reduction/Sub_total_Exceed Fraud    2630
Voucher Fraud                        957
Tip Fraud                            420
No Fraud Category                     27
Tip/Voucher Fraud                     18
Refund Fraud                          11
Reduction/Tip Fraud                    1
Name: count, dtype: int64

In [11]:
de_data.to_csv('data/monthly_data.csv',index=False)

In [12]:
pd.read_csv('data/monthly_data.csv').shape

/tmp/ipykernel_160/895426103.py:1: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('data/monthly_data.csv').shape


(2797471, 63)

In [110]:
%%time

body = s3.get_object(Bucket=bucket_name, Key=file_path)["Body"].read()
string_data = body.decode("utf-8")
de_data = pd.read_csv(
    StringIO(string_data)
)
print(de_data.shape)

(2217722, 64)
CPU times: user 104 µs, sys: 0 ns, total: 104 µs
Wall time: 97.5 µs


In [103]:
data_team_join = full_data.columns

In [107]:
dec_data.difference(data_team_join)

Index(['Unnamed: 0', 'is_fraud'], dtype='object')

In [106]:
data_team_join.difference(dec_data)

Index(['birth_date'], dtype='object')

In [104]:
data_team_join.shape

(63,)

In [ ]:
data_team_join

In [98]:
dec_data = full_data.columns
dec_data

Index(['Unnamed: 0', 'bus_date', 'pos_venue_id', 'order_id', 'device_order_id',
       'time_zone', 'order_local_time', 'sales_hr', 'ord_beg_time',
       'ord_close_time', 'item_count', 'gross_total', 'net_total', 'taxes',
       'exclusive_tax', 'inclusive_tax', 'reduction_amt', 'cust_prof_id',
       'mobile_ord_fl', 'original_order_id', 'vendor_loc_id', 'fee_amt',
       'tip_amount', 'sync_status', 'src_sys_id', 'vendor_id',
       'menu_vendor_id', 'refund_vendor_id', 'refund_venue_id',
       'pos_terminal_id', 'reduction_cd', 'active_fl', 'order_tab_id',
       'order_status_cd', 'united_account_id', 'card_type', 'payment_type',
       'transaction_id', 'transaction_seq_nu', 'cash_recycler_tiny_code',
       'payment_amount', 'account_id', 'payment_amt_rewards_points',
       'voucher_number', 'emp_email', 'cust_first_name', 'cust_last_name',
       'activity_status_code', 'employee_role_id', 'employee_role_name',
       'cust_emply_verified_fl', 'cust_emply_active_fl', 'employ

In [100]:
full_data.shape

(2217722, 64)

In [97]:
full_data.is_fraud

0          0
1          0
2          0
3          0
4          0
          ..
2217717    0
2217718    0
2217719    0
2217720    0
2217721    0
Name: is_fraud, Length: 2217722, dtype: int64

In [93]:
full_data.shape

(2165600, 63)

In [78]:
full_data.united_account_id

0       d41d8cd98f00b204e9800998ecf8427e
1       d41d8cd98f00b204e9800998ecf8427e
2       d41d8cd98f00b204e9800998ecf8427e
3       d41d8cd98f00b204e9800998ecf8427e
4       d41d8cd98f00b204e9800998ecf8427e
                      ...               
4676    d41d8cd98f00b204e9800998ecf8427e
4677    d41d8cd98f00b204e9800998ecf8427e
4678    d41d8cd98f00b204e9800998ecf8427e
4679    d41d8cd98f00b204e9800998ecf8427e
4680    d41d8cd98f00b204e9800998ecf8427e
Name: united_account_id, Length: 4681, dtype: object

In [73]:
full_data.bus_date.min()

'2023-12-01'

In [49]:
# full_data.bus_date.max()

'2023-11-10'

In [38]:
test_data.bus_date.min()

'2023-12-01'

In [36]:
full_data['bus_date'].max()

'2023-11-10'

In [27]:
full_data.shape

(693503, 63)

In [22]:
full_data.shape

(693503, 140)

In [17]:
test_data

,bus_date,pos_venue_id,order_id,device_order_id,time_zone,order_local_time,sales_hr,ord_beg_time,ord_close_time,item_count,...,order_status_cd_FE,payment_type_FE,pos_terminal_id_FE,pos_venue_id_FE,vendor_id_FE,vendor_loc_id_FE,voucher_number_FE,SHAP_features,Detailed_features_with_values,fraud_category
0,2023-12-07,1143,38247542,231207025000459,America/Chicago,2023-12-07 12:18:41,1,2023-12-07 18:18:41,2023-12-07 18:18:41,4,...,0.419614,0.168153,0.425172,0.063989,0.004108,0.000037,0.997984,FE_mean_reduction_amt_for_each_cust_prof_id_AG...,This transaction is predicted fraud with confi...,Reduction/Sub_total_Exceed Fraud
1,2023-12-05,1142,38098658,231205014000190,America/Chicago,2023-12-05 08:25:51,8,2023-12-05 14:25:51,2023-12-05 14:25:51,4,...,0.419614,0.168153,0.425172,0.047688,0.002365,0.000072,0.997984,FE_mean_reduction_amt_for_each_cust_prof_id_AG...,This transaction is predicted fraud with confi...,Reduction/Sub_total_Exceed Fraud
2,2023-12-06,1150,38204792,231207024000022,America/New_York,2023-12-06 19:29:37,1,2023-12-07 00:29:37,2023-12-07 00:29:37,2,...,0.419614,0.168153,0.425172,0.065778,0.004542,0.000086,0.997984,FE_mean_reduction_amt_for_each_cust_prof_id_AG...,This transaction is predicted fraud with confi...,Reduction/Sub_total_Exceed Fraud
3,2023-12-05,1142,38123716,231205031000467,America/Chicago,2023-12-05 15:19:25,1,2023-12-05 21:19:25,2023-12-05 21:19:25,2,...,0.419614,0.168153,0.425172,0.047688,0.002365,0.000042,0.997984,FE_mean_reduction_amt_for_each_cust_prof_id_AG...,This transaction is predicted fraud with confi...,Reduction/Sub_total_Exceed Fraud
4,2023-12-07,1142,38239052,231207012000392,America/Chicago,2023-12-07 10:27:42,1,2023-12-07 16:27:42,2023-12-07 16:27:42,2,...,0.419614,0.168153,0.425172,0.047688,0.004028,0.000012,0.997984,FE_mean_reduction_amt_for_each_cust_prof_id_AG...,This transaction is predicted fraud with confi...,Reduction/Sub_total_Exceed Fraud
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,2023-12-05,1114,38124486,231205026000474,America/New_York,2023-12-05 16:26:11,1,2023-12-05 21:26:11,2023-12-05 21:26:11,1,...,0.419614,0.115086,0.425172,0.034979,0.011093,0.000077,0.997984,FE_cust_prof_id_voucher_number_ct_AG + FE_devi...,This transaction is predicted fraud with confi...,Voucher Fraud
1984,2023-12-03,1143,3800930901,231204015000082,America/Chicago,2023-12-03 19:48:17,1,2023-12-04 01:48:17,2023-12-04 01:48:17,1,...,0.574828,0.022195,0.425172,0.063989,0.003858,0.000087,0.997984,FE_cust_prof_id_bus_date_TB_FE + FE_tip_gt_50_...,This transaction is predicted fraud with confi...,No Fraud Category
1985,2023-12-03,1141,37957543,231203002000229,America/Chicago,2023-12-03 09:08:52,9,2023-12-03 15:08:53,2023-12-03 15:08:53,1,...,0.419614,0.643130,0.425172,0.043581,0.004405,0.000132,0.997984,FE_employee_role_id_reduction_cd_CB + FE_mean_...,This transaction is predicted fraud with confi...,Reduction Fraud
1986,2023-12-08,1141,38359397,231209011000033,America/Chicago,2023-12-08 18:24:41,1,2023-12-09 00:24:41,2023-12-09 00:24:41,3,...,0.419614,0.168153,0.425172,0.043581,0.005745,0.000109,0.997984,FE_employee_role_id_reduction_cd_CB + FE_mean_...,This transaction is predicted fraud with confi...,Reduction Fraud
